In [19]:
!pip install mediapipe opencv-python matplotlib



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Load Video & Initialize MediaPipe

In [4]:
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture("demo video.mp4")  # Replace with your video file


# Track Landmarks

In [6]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    cv2.imshow("Exercise", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Define Rule-Based Repetition Counter

In [7]:
counter = 0
stage = None  # "down" or "up"

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
        knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y

        # Rule: squat = hip below knee -> then back up
        if hip > knee and stage != "down":
            stage = "down"
        if hip < knee and stage == "down":
            stage = "up"
            counter += 1

        cv2.putText(frame, f"Reps: {counter}", (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

        mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow("Exercise", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
